In [1]:
import random
import importlib
import gp.generate
import gp.selection
import gp.crossover
import gp.mutation
import gp.formulate_coding_AST
import gp.binance_trade_client
import gp.mapping_columns_name
import gp.fitness_evaluate

importlib.reload(gp.fitness_evaluate)
importlib.reload(gp.generate)
importlib.reload(gp.selection)
importlib.reload(gp.crossover)
importlib.reload(gp.mutation)
importlib.reload(gp.formulate_coding_AST)
importlib.reload(gp.binance_trade_client)
importlib.reload(gp.mapping_columns_name)

<module 'gp.mapping_columns_name' from 'c:\\Users\\QinTian\\Desktop\\F\\gp\\mapping_columns_name.py'>

In [2]:
#从binance接口下载数据
df=gp.binance_trade_client.data_loader(symbol='BTCUSDT',interval='1h',back_days=1000)
df['future_return']=df['close'].pct_change().shift(-1)
df=df.dropna()
df.tail(5)

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,future_return
23993,2024-11-14 12:00:00,91491.85,91599.99,90945.00,91243.17,1587.33383,2024-11-14 12:59:59.999,1.448928e+08,451170,734.14002,6.703008e+07,-0.005357
23994,2024-11-14 13:00:00,91243.17,91380.00,90559.65,90754.42,2037.91332,2024-11-14 13:59:59.999,1.853306e+08,446427,800.67679,7.281210e+07,-0.022416
23995,2024-11-14 14:00:00,90754.42,91790.00,88639.76,88720.03,5271.45761,2024-11-14 14:59:59.999,4.751695e+08,897010,2406.96396,2.172742e+08,-0.008702
23996,2024-11-14 15:00:00,88720.03,89638.50,87829.57,87947.99,6221.33894,2024-11-14 15:59:59.999,5.517983e+08,1076245,2806.89051,2.490308e+08,0.014217
23997,2024-11-14 16:00:00,87947.99,89239.79,87526.74,89198.31,3863.10102,2024-11-14 16:59:59.999,3.411234e+08,602093,2118.87550,1.871513e+08,0.000455


In [10]:
polulation_num=10000
echo=20
#产生初始种群,数量为10000
population = gp.generate.generate(low=1,high=2,num=polulation_num)

#映射成可以被计算的形式
population=gp.mapping_columns_name.trans_(population )
print(population[:3])
#解析成算法树
population=[gp.formulate_coding_AST.ExpressionTree(i) for i in population]

['add(low, high)', 'sub(ts_pct_change(low,1), low)', 'mul(ts_pct_change(taker_buy_base_asset_volume,1), ts_pct_change(close,1))']


In [11]:
fitness_score=[]
count=0
from IPython.display import clear_output
from tqdm import tqdm
import time
import warnings

# 忽略所有警告
warnings.filterwarnings("ignore")
for count in range(echo):
    print(f'第{count+1}次演化')
    #计算适应值
    for i in tqdm(population,desc='计算适应值'):
              
            #   factor_name=gp.formulate_coding_AST.f_coding(i.root,df,)
              i_expr=gp.mapping_columns_name.trans_(str(i))
              tree=gp.formulate_coding_AST.ExpressionTree(i_expr)
              factor_name=gp.formulate_coding_AST.f_coding_printfree(tree.root,df,)
              if factor_name:
                _, detail=gp.fitness_evaluate.evaluate_factor_with_score(df, factor_col= factor_name, future_return_col='future_return', quantiles=5, weights=None)
                df.drop(factor_name, axis=1, inplace=True)#防止df过大溢出
                fitness_score.append(detail['max_min_group_return_diff'])
              else:
                #print(i)
                fitness_score.append(0)
            #   clear_output(wait=True)
    #自然选择环节
    reserve_group=gp.selection.reserve_top_n_individuals(population,fitness_score,int(polulation_num*0.2))
    slection_group=gp.selection.tournament_selection(population, fitness_score, num_to_select=int(polulation_num*0.8), tournament_size=3)
    average = sum(fitness_score) /len(fitness_score)
    maximu=max(fitness_score)
    print(f"最大适应度是: {maximu}")
    print(f"平均适应度是: {average}")
    fitness_score=[] #fitness_score重置为空
    #交叉环节
    random.shuffle(population)#打乱种群
    crossover_group=gp.crossover.population_crossover(slection_group[:int(polulation_num*0.4)])
    slection_group=crossover_group+slection_group[int(polulation_num*0.4):]
    #变异环节
    slection_group=gp.mutation.population_mutation(slection_group, mutation_rate=0.1,low=1,high=2)
    population=slection_group+reserve_group
    s=0
    for i in population:
         s+=i.get_depth()
    print(f"种群平均深度是: {s/polulation_num}")
    
   
   
    

    




第1次演化


计算适应值: 100%|██████████| 10000/10000 [03:01<00:00, 55.11it/s]


最大适应度是: 0.0004347650785474789
平均适应度是: 0.00017032509976627093
种群平均深度是: 2.503
第2次演化


计算适应值: 100%|██████████| 10000/10000 [02:48<00:00, 59.49it/s]


最大适应度是: 0.0006533965020346248
平均适应度是: 0.00018790537684556299
种群平均深度是: 2.6539
第3次演化


...

最大适应度是: 0.0023156116924329916
平均适应度是: 0.0001508533550089507
种群平均深度是: 4.0876
第7次演化


计算适应值: 100%|██████████| 10000/10000 [05:11<00:00, 32.07it/s]


最大适应度是: 0.017753054247832502
平均适应度是: 0.00017850129518081
种群平均深度是: 5.1598
第8次演化


计算适应值:   9%|▉         | 901/10000 [00:34<05:45, 26.31it/s]


KeyboardInterrupt: 

In [9]:
print(population[10])

mul(div(div(ts_pct_change(open, 1), ts_pct_change(taker_buy_quote_asset_volume, 100)), add(open, ts_pct_change(taker_buy_quote_asset_volume, 100))), volume)


In [ ]:
gp.formulate_coding_AST.f_coding('sub(div(volume, ts_pct_change(close,1)), mul(volume, ts_pct_change(volume,1)))',df,)

In [ ]:
df

In [ ]:
b=[random.random() for i in range(100)]

In [ ]:
m,n=gp.crossover.crossover(a[0],a[1])
print(m,n)

In [ ]:
print(gp.formulate_coding_AST.ExpressionTree(m))

In [ ]:
print(gp.formulate_coding_AST.ExpressionTree(n))

In [ ]:
selected_individuals=gp.selection.tournament_selection(formula, random_list, num_to_select=100, tournament_size=3)
selected_individuals